# GGUF Quantization Guide

**Duration:** ~20 min | **Platform:** Kaggle dual Tesla T4

This notebook covers GGUF quantization types, how to inspect GGUF files,
VRAM planning for Kaggle T4 GPUs, and benchmarking different quantizations.

### Quantization Overview

| Family | Types | Bits | Quality | Speed |
|--------|-------|------|---------|-------|
| **Legacy** | Q4_0, Q8_0 | 4-8 | Low-High | Fast |
| **K-Quants** | Q3_K_S … Q6_K | 3-6 | Good | Balanced |
| **I-Quants** | IQ3_XS … IQ4_NL | 3-4 | Best-at-size | Slower |

In [ ]:
!pip install -q git+https://github.com/llamatelemetry/llamatelemetry.git@v1.0.0

import llamatelemetry
llamatelemetry.init(service_name="gguf-quantization")

## Understanding Quantization Types

The SDK provides `QUANT_TYPE_INFO` with metadata for every GGML quantization type,
including bits-per-weight and recommended use cases.

In [ ]:
from llamatelemetry.api import GGMLType

# Common quantization types and their properties
quant_info = [
    ("Q4_0",    4.0,  "Legacy 4-bit, fast but lower quality"),
    ("Q4_K_M",  4.5,  "K-Quant 4-bit medium — best balance for most uses"),
    ("Q5_K_M",  5.5,  "K-Quant 5-bit medium — higher quality, +20% VRAM"),
    ("Q6_K",    6.5,  "K-Quant 6-bit — near-FP16 quality"),
    ("Q8_0",    8.0,  "8-bit — highest quality quantized"),
    ("IQ3_XS",  3.3,  "I-Quant 3-bit — smallest, for 70B on dual T4"),
    ("IQ4_NL",  4.5,  "I-Quant 4-bit — better quality than Q4_K_M at same size"),
]

print(f"{'Type':<10} {'Bits/W':<8} {'7B Size (GB)':<14} {'13B Size (GB)':<14} Description")
print("-" * 80)
for name, bpw, desc in quant_info:
    size_7b = 7.0 * bpw / 8
    size_13b = 13.0 * bpw / 8
    print(f"{name:<10} {bpw:<8.1f} {size_7b:<14.1f} {size_13b:<14.1f} {desc}")

## Inspecting GGUF Files

Use `parse_gguf_header()` to read architecture metadata, layer counts, and
quantization details directly from a GGUF file.

In [ ]:
from huggingface_hub import hf_hub_download
from llamatelemetry.llama import parse_gguf_header, get_model_summary
from llamatelemetry.models import ModelInfo

model_path = hf_hub_download(
    repo_id="bartowski/google_gemma-3-1b-it-GGUF",
    filename="google_gemma-3-1b-it-Q4_K_M.gguf",
    cache_dir="/root/.cache/huggingface",
)

# Parse GGUF header
info = parse_gguf_header(model_path)
print(f"File: {info.file}")
print(f"Size: {info.size_mb:.1f} MB")
print(f"Architecture: {info.metadata.architecture}")
print(f"Context length: {info.metadata.context_length}")
print(f"Embedding dim: {info.metadata.embedding_length}")
print(f"Layers: {info.metadata.block_count}")

# One-line summary
print(f"\nSummary: {get_model_summary(model_path)}")

## Kaggle T4 VRAM Recommendations

Each T4 has 15 GB of VRAM. With dual T4, you have 30 GB total.
The model must fit in VRAM plus overhead for context and KV cache.

In [ ]:
# VRAM calculator for Kaggle T4
T4_VRAM_GB = 15.0
DUAL_T4_VRAM_GB = 30.0
KV_OVERHEAD_GB = 1.5  # approximate KV cache overhead

models = [
    ("Gemma-3 1B",   1,  "Q4_K_M", 4.5),
    ("Gemma-3 4B",   4,  "Q4_K_M", 4.5),
    ("Llama-3.2 7B", 7,  "Q4_K_M", 4.5),
    ("Llama-3.2 7B", 7,  "Q5_K_M", 5.5),
    ("Llama-3.1 13B", 13, "Q4_K_M", 4.5),
    ("Llama-3.1 70B", 70, "IQ3_XS", 3.3),
    ("Llama-3.1 70B", 70, "Q4_K_M", 4.5),
]

print(f"{'Model':<20} {'Quant':<10} {'Size (GB)':<12} {'Single T4':<12} {'Dual T4'}")
print("-" * 70)
for name, params, quant, bpw in models:
    size_gb = params * bpw / 8
    fits_single = "Yes" if (size_gb + KV_OVERHEAD_GB) <= T4_VRAM_GB else "No"
    fits_dual = "Yes" if (size_gb + KV_OVERHEAD_GB) <= DUAL_T4_VRAM_GB else "No"
    print(f"{name:<20} {quant:<10} {size_gb:<12.1f} {fits_single:<12} {fits_dual}")

## Benchmarking Different Quantizations

Compare inference speed and output quality across quantization levels.

In [ ]:
import time
from llamatelemetry.llama import ServerManager, LlamaCppClient

# Download Q5_K_M for comparison
model_q5 = hf_hub_download(
    repo_id="bartowski/google_gemma-3-1b-it-GGUF",
    filename="google_gemma-3-1b-it-Q5_K_M.gguf",
    cache_dir="/root/.cache/huggingface",
)

test_prompt = "Explain the theory of relativity in simple terms."
quant_models = [("Q4_K_M", model_path), ("Q5_K_M", model_q5)]

for quant_name, mpath in quant_models:
    mgr = ServerManager()
    mgr.start_server(model_path=mpath, gpu_layers=99, ctx_size=2048)
    mgr.wait_until_ready(timeout=60)
    client = LlamaCppClient(base_url="http://127.0.0.1:8090")

    # Warm up
    client.chat.completions.create(messages=[{"role": "user", "content": "Hi"}], max_tokens=8)

    # Benchmark
    t0 = time.perf_counter()
    resp = client.chat.completions.create(
        messages=[{"role": "user", "content": test_prompt}],
        max_tokens=128, temperature=0.7,
    )
    elapsed = time.perf_counter() - t0
    tokens = resp.usage.completion_tokens

    mem = llamatelemetry.gpu.snapshot()
    vram_mb = sum(s.mem_used_mb for s in mem)

    print(f"\n--- {quant_name} ---")
    print(f"  VRAM: {vram_mb} MB total across GPUs")
    print(f"  Latency: {elapsed*1000:.0f} ms")
    print(f"  Throughput: {tokens/elapsed:.1f} tok/s")
    print(f"  Output: {resp.choices[0].message.content[:120]}...")

    mgr.stop_server()
    time.sleep(2)

## I-Quants for Large Models

I-Quants (IQ3_XS, IQ3_S, IQ4_NL) use importance-weighted quantization for
better quality at extreme compression. This makes 70B models feasible on dual T4.

In [ ]:
# I-Quant VRAM estimations for large models
large_models = [
    ("70B IQ3_XS",  70, 3.3, "Best fit for dual T4 — 28.9 GB"),
    ("70B IQ3_S",   70, 3.5, "Slightly better quality — requires tight VRAM"),
    ("70B Q4_K_M",  70, 4.5, "Does NOT fit on dual T4 (39.4 GB)"),
    ("34B Q4_K_M",  34, 4.5, "Comfortable fit on dual T4"),
    ("34B Q5_K_M",  34, 5.5, "Fits with reduced context window"),
]

print(f"{'Model':<15} {'VRAM (GB)':<12} {'Fits Dual T4?':<15} Notes")
print("-" * 70)
for name, params, bpw, notes in large_models:
    vram = params * bpw / 8
    fits = "Yes" if (vram + KV_OVERHEAD_GB) <= DUAL_T4_VRAM_GB else "No"
    print(f"{name:<15} {vram:<12.1f} {fits:<15} {notes}")

## Quick Reference

**Kaggle T4 Recommendations:**

| Model Size | Recommended Quant | GPUs | Context |
|-----------|-------------------|------|--------|
| 1B-4B | Q5_K_M or Q6_K | Single T4 | 4096+ |
| 7B-8B | Q4_K_M | Single or Dual T4 | 2048-4096 |
| 13B | Q4_K_M | Dual T4 (50/50) | 2048 |
| 34B | Q4_K_M | Dual T4 (50/50) | 1024-2048 |
| 70B | IQ3_XS | Dual T4 (50/50) | 512-1024 |

In [ ]:
llamatelemetry.shutdown()
print("Done.")